In [1]:
%cd ..

/home/eli/AnacondaProjects/epych


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych

In [4]:
%matplotlib inline

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [7]:
NWB_SUBJECTS = glob.glob('/mnt/data/DRAFT/000253/sub-*/')

In [8]:
PILOT_FILES = []

In [9]:
def samplings():
    for cond in CONDITIONS:
        for s, subject_dir in enumerate(sorted(NWB_SUBJECTS)):
            subject = subject_dir.split('/')[-2]
            if not os.path.exists(subject_dir + "/" + cond):
                continue
            sampling = epych.recording.Sampling.unpickle(subject_dir + "/" + cond).smap(lambda sig: sig.select_channels(["VIS" in loc for loc in sig.channels.location]))
            yield sampling
            del sampling
            logging.info("Loaded LFPs for %s in subject %s" % (cond, subject))
        logging.info("Loaded LFPs for %s" % cond)

In [10]:
alignment = epych.statistics.alignment.AlignmentSummary()

In [11]:
alignment.calculate(samplings())

INFO:root:Loaded LFPs for go_gloexp in subject sub-621890
INFO:root:Loaded LFPs for go_gloexp in subject sub-632485
INFO:root:Loaded LFPs for go_gloexp in subject sub-632487
INFO:root:Loaded LFPs for go_gloexp in subject sub-637542
INFO:root:Loaded LFPs for go_gloexp in subject sub-637908
INFO:root:Loaded LFPs for go_gloexp in subject sub-637909
INFO:root:Loaded LFPs for go_gloexp in subject sub-640507
INFO:root:Loaded LFPs for go_gloexp in subject sub-642507
INFO:root:Loaded LFPs for go_gloexp in subject sub-645322
INFO:root:Loaded LFPs for go_gloexp in subject sub-645324
INFO:root:Loaded LFPs for go_gloexp in subject sub-645495
INFO:root:Loaded LFPs for go_gloexp in subject sub-647836
INFO:root:Loaded LFPs for go_gloexp in subject sub-649323
INFO:root:Loaded LFPs for go_gloexp in subject sub-649324
INFO:root:Loaded LFPs for go_gloexp
INFO:root:Loaded LFPs for go_seqctl in subject sub-621890
INFO:root:Loaded LFPs for go_seqctl in subject sub-632485
INFO:root:Loaded LFPs for go_seqctl 

{'VISam': <epych.statistics.alignment.LaminarAlignment at 0x70090b51d790>,
 'VISpm': <epych.statistics.alignment.LaminarAlignment at 0x700736304fd0>,
 'VISp': <epych.statistics.alignment.LaminarAlignment at 0x700734b12bd0>,
 'VISl': <epych.statistics.alignment.LaminarAlignment at 0x700734b33d50>,
 'VISrl': <epych.statistics.alignment.LaminarAlignment at 0x700734b42b10>,
 'VIS': <epych.statistics.alignment.LaminarAlignment at 0x700734b6fad0>,
 'VISal': <epych.statistics.alignment.LaminarAlignment at 0x700734b6f6d0>}

In [12]:
alignment.pickle("/mnt/data/DRAFT/000253/visual_alignment")